In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
%matplotlib inline
import time

In [ ]:
DATA = pd.read_csv("/data/zkp/buy2/service/DATA.csv")
DATA_X = np.load("/data/zkp/buy2/service/DATA.npy")

# DATA_X = DATA_X[(DATA.container_ip=="33.61.250.148")]
# DATA = DATA[(DATA.container_ip=="33.61.250.148")]

X = []

In [ ]:
DATA

In [ ]:
mp = {}
cpu = offline.cpu_util__pct.values
t = offline.sample_time.values
for i in range(offline.shape[0]):
    mp[t[i]] = [cpu[i]]

In [ ]:
for ip in svc.container_ip.unique():
    temp = svc[svc.container_ip==ip]
    t = temp.sample_time__m.apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S")))).values
    r = temp["rank"].values
    for i in range(t.shape[0]):
        if t[i] in mp:
            mp[t[i]].append(r[i])


In [ ]:
mpp = []
for key in mp:
    if len(mp[key]) == 6:
        mpp.append(mp[key])
#         X.append(DATA_X[DATA.sample_time==key])
mpp = np.asarray(mpp)

In [ ]:
# X = np.asarray(X).reshape(-1, 60)
X.shape, mpp.shape

In [ ]:
for i in range(1, 6):
    mpp[:, i] = (mpp[:, i] - np.mean(mpp[:, i])) / np.std(mpp[:, i])

In [ ]:
plt.plot(range(mpp[:, 1].shape[0]), mpp[:, 1])

In [ ]:
mean = []
for ip in svc.container_ip.unique():
    temp = svc[svc.container_ip==ip]
    mean.append(np.mean(temp.rt__ms_Q.values))


In [ ]:
svc.container_ip.unique()

In [ ]:
np.corrcoef(mpp.T)

In [ ]:
y = []
for i in range(mpp.shape[0]):
    y.append(mpp[i, 1]*0.515*16 + mpp[i, 3]*0.256*4 + mpp[i, 4]*0.323*8)

In [ ]:
y = np.asarray(y)

In [ ]:
y = (y - min(y)) / (max(y) - min(y))

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
model.score(X, y)

In [ ]:
predict = model.predict(X)
plt.scatter(predict, y)
plt.xlabel("Predict score")
plt.ylabel("Machine level interference")

In [ ]:
pod_meta  = pd.read_csv("/data/zkp/pod_meta.csv")
pod_meta

In [ ]:
svc = pd.read_csv("/data/zkp/buy2/service/allservice2.csv")
svc = svc[(svc.throughput_type=="HSF") &(~pd.isna(svc.rt__ms_Q))]

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/service/onoffline2.csv")
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
# DATA = pd.merge(DATA, pod_meta[["container_ip", "container_app_group_name"]], on=["container_ip"], how="left")
DATA = pd.read_csv("/data/zkp/buy2/service/allservice3.csv")

In [ ]:
DATA = DATA.sort_values(by=["container_app_group_name"])

In [ ]:
data = DATA[DATA.sample_time<1644422400]
data = data[["container_app_group_name", "rt__ms_Q", "CPU_time__s"]]
data = data.sort_values(by=["container_app_group_name"])
data

In [ ]:
corr1 = {}
for app in data.container_app_group_name.unique():
    temp = data[(data.container_app_group_name==app) & (~pd.isna(data.CPU_time__s))]
    if temp.shape[0] < 100:
        corr1[app] = 0
        continue
    c1 = temp.rt__ms_Q.corr(temp.CPU_time__s)
    if np.isnan(c1):
        c1 = 0
    corr1[app] = c1

In [ ]:
corr1["ump2_detailhost"], corr2["ump2_detailhost"]

In [ ]:
rtP1 = {}
for app in data.container_app_group_name.unique():
    temp = data[data.container_app_group_name==app]
    if temp.shape[0] < 100:
        rtP1[app] = 0
        continue
#     rtP1[app] = np.std(temp.rt__ms_Q.values) / np.mean(temp.rt__ms_Q.values) if np.mean(temp.rt__ms_Q.values) else 0
    rtP1[app] = np.percentile(temp.rt__ms_Q.values, 95) / np.percentile(temp.rt__ms_Q.values, 50) if np.percentile(temp.rt__ms_Q.values, 50) else 0

In [ ]:
score = []
for app in DATA.container_app_group_name.unique():
    temp = data[data.container_app_group_name==app].rt__ms_Q.values
    if temp.shape[0] < 10:
        for i in range(DATA[DATA.container_app_group_name==app].shape[0]):
            score.append(0)
        continue
    p = []
    for i in range(10, 101, 10):
        p.append(np.percentile(temp, i))
    rt = DATA[DATA.container_app_group_name==app].rt__ms_Q.values
    for i in rt:
        for j in range(1, 11):
            if i <= p[j-1] or j == 10:
                score.append(j)
                break
DATA["score"] = score    

In [ ]:
# co = []
# rtP = []
rtPP = []
for app in DATA.container_app_group_name.values:
    if app in rtP1:
#         co.append(corr1[app])
#         rtP.append(rtP1[app])
        rtPP.append(rtP1[app])
    else:
#         co.append(0)
#         rtP.append(0)
        rtPP.append(0)
# DATA["co"] = co
DATA["rtPP"] = rtPP

In [ ]:
buy2 = pd.read_csv("/data/zkp/buy2/service/DATA.csv")
buy2_X = np.load("/data/zkp/buy2/service/DATA.npy")

In [ ]:
temp = []
for x in buy2_X:
    temp.append(x)
buy2["X"] = temp

In [ ]:
buy2 = pd.merge(buy2, pod_meta[["nc_sn", "container_ip"]], on=["container_ip"], how="left")
DATA = buy2
# DATA = pd.merge(DATA, buy2[["nc_sn", "sample_time", "X"]], on=["nc_sn", "sample_time"], how="left")

In [ ]:
DATA

In [ ]:
X = [[], [], [], []]
y = [[], [], [], []]
for nc in data.nc_sn.unique():
    y_nc = []
    X_nc = []
    temp = data[data.nc_sn==nc]
    for t in temp.sample_time.unique():
        tep = temp[temp.sample_time==t]
        y_nc.append([np.sum(tep["score"].values),
                     np.sum(tep["score"].values*tep.request_cpu.values),
                     np.sum(tep["score"].values*tep.rt__ms_Q.values),
                     np.sum(tep["rank"].values*tep["corr"].values),
                    ])
        X_nc.append(tep.X.values[0])
    y_nc = np.asarray(y_nc)
    X_nc = np.asarray(X_nc)
    for i in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X_nc, y_nc[:, [i]])
        scaler = MinMaxScaler().fit(y_train)
        y_test = scaler.transform(y_test)
        for j in y_test:
            y[i].append(j[0])
        for j in X_test:
            X[i].append(j)

In [ ]:
X = np.asarray(X).T
y = np.asarray(y).T

In [ ]:
X.shape, y.shape

In [ ]:
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y[:, i])
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    
    predict = model.predict(X_test)
    plt.scatter(predict[::50], y_test[::50])
    plt.xlabel("Predict score")
    plt.ylabel("Machine level interference")
    plt.show()

    error = abs(predict - y_test)
    plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))
    plt.xlim(0, 1)
    plt.xlabel("error", fontsize=30)
    plt.ylabel("CDF", fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.grid(":")
    plt.show()

In [ ]:
P = [[], [], [], []]
T = [[], [], [], []]
for nc in data.nc_sn.unique():
#     if nc != "i-8vb8e59yufyqspqng372":
#         continue
    X = []
    y = []
    temp = data[data.nc_sn==nc]
    for t in temp.sample_time.unique():
        tep = temp[temp.sample_time==t]
        y.append([np.sum(tep["score"].values),
                  np.sum(tep["score"].values*tep.request_cpu.values),
                  np.sum(tep["score"].values*tep["co"].values),
                  np.sum(tep["score"].values*tep["rtPP"].values),
                  ])
        X.append(tep.X.values[0])
    X = np.asarray(X)
    if X.shape[0] < 1000:
        continue
    y = np.asarray(y)
    for index in range(4):
        p = []
        for i in range(10, 101, 10):
            p.append(np.percentile(y[:, index], i))
        y_new = []
        for i in y[:, index]:
            for j in range(1, 11):
                if i <= p[j-1]:
                    y_new.append(j)
                    break
        y[:, index] = np.asarray(y_new)
#         y[:, index] = (y[:, index] - np.percentile(y[:, index], 1)) / (np.percentile(y[:, index], 99) - np.percentile(y[:, index], 1))

    print(nc, end=' ')
    for index in range(4):
        model = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X, y[:, index])
#         scaler = MinMaxScaler().fit(y_train)
#         y_train = scaler.transform(y_train)
#         y_test = scaler.transform(y_test)
        model.fit(X_train, y_train)
        print(model.score(X_test, y_test), end='\t')
        predict = model.predict(X_test)
    #     plt.scatter(predict, y)
    #     plt.xlabel("Predict score")
    #     plt.ylabel("Machine level interference")
    #     plt.show()
#         e = abs(predict - y_test)
#         for i in e:
        P[index].append(predict)
        T[index].append(y_test)
    print()
#     plt.scatter(predict, scaler.transform(y[:, [index]]))
#     plt.xlabel("predict")
#     plt.ylabel("interference score")

In [ ]:
MODEL = {}
P = {}

In [ ]:
for nc in data.nc_sn.unique():
    X = []
    y = []
    temp = data[data.nc_sn==nc]
    sample_time = temp.sample_time.unique()
    for t in sample_time:
        tep = temp[temp.sample_time==t]
        y.append(np.sum(tep["score"].values*tep["co"].values))
        X.append(tep.X.values[0])
    X = np.asarray(X)
    if X.shape[0] < 1000:
        continue
    y = np.asarray(y)
    p = []
    for i in range(10, 101, 10):
        p.append(np.percentile(y, i))
    for i in range(y.shape[0]):
        for j in range(1, 11):
            if y[i] <= p[j-1]:
                y[i] = j
                break

    print(nc, end=' ')
    model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = X[sample_time<1644505200], X[sample_time>=1644505200],  y[sample_time<1644505200], y[sample_time>=1644505200]
    model.fit(X_train, y_train)
    MODEL[nc] = model
    P[nc] = model.predict(X_test)
    print(model.score(X_test, y_test), end='\t')

1644465600: 0.1596676737160121
+3600: 0.14778449144008057
+3600: 0.2510255560934594
+3600: 0.13641473759517606
+3600: 0.051544288301155344
-3600: 0.34655085599194363
-3600: 0.2919436052366566
-3600: 0.283031218529708
-3600: 0.39798590130916417
-3600: 0.4077794561933535
-3600: 0.7136203423967774
-3600: 0.7107754279959718
-3600: 0.7244712990936556
-3600: 0.7406847935548841

In [ ]:
1644498000, 1644422400, 1644440400

In [ ]:
TIMESTAMP = 1644422400

In [ ]:
for i in range(0, 3600*24, 3600):
    TIMESTAMP = 1644422400 + i
    print(TIMESTAMP, end=', ')
    base = DATA[(DATA.sample_time>TIMESTAMP) & (DATA.sample_time<TIMESTAMP + 3600)]
    print(base.machine_cpu.mean(), base.machine_cpu.std(), end=', ')
    # temp = data
    result = []
    for i in e_score:
        if i in MODEL:
            X = []
            for x in base[base.nc_sn==i].X.values:
                 X.append(x)

            for j in MODEL[i].predict(np.asarray(X)).tolist():
                result.append(j)

    base = np.bincount(result, minlength=11)
    SUM = 0
    for i in range(base.shape[0]):
        SUM += base[i] * i
    print(SUM / np.sum(base))

In [ ]:
base = np.asarray(result).reshape(-1, 59)

In [ ]:
np.savetxt("/data/zkp/figure/simulation/base2_score.txt", result)

In [ ]:
result.shape

In [ ]:
batch = pd.read_csv("/data/zkp/buy2/instance.csv")

In [ ]:
batch

In [ ]:
batch["job_task_name"] = batch.fuxi_job_name + batch.fuxi_task_name
batch["total_time"] = batch.end_time - batch.start_time
G = batch[["job_task_name", "fuxi_avg_cpu", "fuxi_avg_mem", "total_time"]].groupby(["job_task_name"]).mean()

In [ ]:
task_type = {}
type_cnt = [0 for i in range(60)]
cpu = G.fuxi_avg_cpu.values
mem = G.fuxi_avg_mem.values
t = G.total_time.values
key = G.index.values
for i in range(G.shape[0]):
    if np.isnan(cpu[i]) or np.isnan(mem[i]) or np.isnan(t[i]):
        continue
    index = 0
    if cpu[i] < 50:
        index = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2

    if mem[i] < 512:
        index = (index*5) + 0
    elif 512 <= mem[i] < 1536:
        index = (index*5) + 1
    elif 1536 <= mem[i] < 3072:
        index = (index*5) + 2
    elif 3072 <= mem[i] < 6144:
        index = (index*5) + 3
    else:
        index = (index*5) + 4

    if t[i] < 20:
        index = (index*4) + 0
    elif 20 <= t[i] < 40:
        index = (index*4) + 1
    elif 40 <= t[i] < 100:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
    task_type[key[i]] = index
    type_cnt[index] += 1

In [ ]:
temp = []
for i in batch.job_task_name.values:
    if i in task_type:
        temp.append(int(task_type[i]))
    else:
        temp.append(np.nan)
batch["type"] = temp

In [ ]:
batch = batch[~np.isnan(batch.type)]
batch = batch[~((batch.start_time==0) | (batch.end_time==0))]
batch = batch[~pd.isna(batch.worker_address)]
batch.type = np.asarray(batch.type, dtype="int")

In [ ]:
queue = batch[(batch.start_time>=TIMESTAMP) & (batch.start_time<TIMESTAMP+3600)]

In [ ]:
queue

In [ ]:
data = DATA[["nc_sn", "sample_time", "X"]]
data = data.dropna()
data = data[(data.sample_time>=TIMESTAMP) & (data.sample_time<TIMESTAMP+3600)]

In [ ]:
data

In [ ]:
machine = pd.read_csv("/data/zkp/buy2/machine.csv")

In [ ]:
machine

In [ ]:
data = pd.merge(data, machine[["nc_sn", "sample_time", "machine_cpu"]], on=["nc_sn", "sample_time"], how="left")

In [ ]:
data = data[~pd.isna(data.machine_cpu)]

In [ ]:
data = data[~((data.nc_sn=="i-8vbfjjotg3mxpjcy0xyx") | (data.nc_sn=="i-8vb6iy0d1bzjh4ob84l5") | (data.nc_sn=="i-8vbbuc81w02wwl188bo6"))]
# data = data[~(data.nc_sn=="i-8vbbuc81w02wwl188bo6")]
# data = data[~(data.nc_sn.isin(["i-8vb6iy0d1bzjh4ob84l5", "i-8vbbuc81w02wwl188bo6"]))]

In [ ]:
nc_sn = data[data.sample_time==TIMESTAMP].nc_sn.unique()
nc_sn.shape

In [ ]:
pod_meta = pd.read_csv("/data/zkp/pod_meta.csv")

In [ ]:
ay49c = pod_meta[(pod_meta.container_app_name=="ay49c") & (pod_meta.nc_sn.isin(nc_sn))]

In [ ]:
workermap = {}
for ip, nc in zip(ay49c.container_ip.values, ay49c.nc_sn.values):
    workermap[ip] = nc

In [ ]:
len(workermap)

In [ ]:
queue = queue[queue.worker_address.isin(ay49c.container_ip.values)]

In [ ]:
queue.worker_address.unique().shape

In [ ]:
temp = []
for i in queue.worker_address.unique():
    temp.append(workermap[i])
for i in nc_sn:
    if i not in temp:
        print(i)

In [ ]:
data = data[data.nc_sn.isin(nc_sn)]

In [ ]:
queue = queue[~pd.isna(queue.fuxi_avg_cpu)]

In [ ]:
data.sort_values(by=["sample_time"], inplace=True)

In [ ]:
queue.sort_values(by=["start_time", "end_time"], inplace=True)

In [ ]:
data

In [ ]:
queue

In [ ]:
temp = data[data.sample_time==TIMESTAMP]
BED = {}
for i in temp.nc_sn.unique():
    BED[i] = temp[temp.nc_sn==i].X.values[0].tolist()

In [ ]:
BED

In [ ]:
cur_machine = data[data.sample_time==TIMESTAMP].nc_sn.values
machine_to_index = {}
for i in range(cur_machine.shape[0]):
    machine_to_index[cur_machine[i]] = i

In [ ]:
e_score = {}
e_macpu = {}
e_BED = {}
for key in BED:
    e_score[key] = []
    e_macpu[key] = []
    e_BED[key] = []

In [ ]:
end_list = {}
before = batch[(batch.start_time<TIMESTAMP) & (batch.end_time>TIMESTAMP - 60)].sort_values(by=["start_time", "end_time"])
end_time = before.end_time.values
ip = before.worker_address.values
task_type = before.type.values

for i in range(before.shape[0]):
    if ip[i] not in workermap:
        continue
    if end_time[i] not in end_list:
        end_list[end_time[i]] = {}
    if workermap[ip[i]] not in end_list[end_time[i]]:
        end_list[end_time[i]][workermap[ip[i]]] = []
    end_list[end_time[i]][workermap[ip[i]]].append(task_type[i])
    

In [ ]:
cpu_delta = {}
before = batch[(batch.start_time<TIMESTAMP) & (batch.end_time>TIMESTAMP)].sort_values(by=["start_time", "end_time"])
end_time = before.end_time.values
ip = before.worker_address.values
ins_cpu = before.fuxi_avg_cpu.values

for i in range(before.shape[0]):
    if np.isnan(ins_cpu[i]) or ip[i] not in workermap:
        continue
    if end_time[i] not in cpu_delta:
        cpu_delta[end_time[i]] = np.zeros(cur_machine.shape[0])

    cpu_delta[end_time[i]][machine_to_index[workermap[ip[i]]]] += ins_cpu[i]/100

In [ ]:
cur_score = []
for key in cur_machine:
    cur_score.append(MODEL[key].predict(np.asarray(BED[key]).reshape(1, -1))[0])

In [ ]:
import time
cur_min = TIMESTAMP
start_time = queue.start_time.values
ins_cpu = queue.fuxi_avg_cpu.values
end_time = queue.end_time.values
task_type = queue.type.values 

cur_time = TIMESTAMP

cur_cpu = data[data.sample_time==cur_min].machine_cpu.values

random_list = np.arange(cur_machine.shape[0])

s = time.time()

for i in range(queue.shape[0]):
#     if i%10000 == 0:
#         print(i, end=',')
    if start_time[i] != cur_time:
        while cur_time < start_time[i]:
            if cur_time in end_list:
                for key in end_list[cur_time]:
                    for ins in end_list[cur_time][key]:
                        BED[key][ins] -= 1
            if cur_time in cpu_delta:
                    cur_cpu -= cpu_delta[cur_time]
            cur_time += 1
#         for nc in range(cur_machine.shape[0]):
#             cur_score[nc] = MODEL[cur_machine[nc]].predict(np.asarray(BED[cur_machine[nc]]).reshape(1, -1))[0]
#         print(np.mean(cur_score))
    if start_time[i] >= cur_min + 60:
        cur_min += 60
        print(cur_min, end=',')
        
        for nc in range(cur_machine.shape[0]):
            e_score[cur_machine[nc]].append(MODEL[cur_machine[nc]].predict(np.asarray(BED[cur_machine[nc]]).reshape(1, -1))[0])
#             e_BED[cur_machine[nc]].append(BED[cur_machine[nc]])
            e_macpu[cur_machine[nc]].append(cur_cpu[nc])

#     index = np.argmin(cur_cpu)
    index = np.argsort(cur_cpu)[:10]
    X = []
    for key in index:
        BED[cur_machine[key]][task_type[i]] += 1
        a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
        X.append(a)
        BED[cur_machine[key]][task_type[i]] -= 1
        if a == 1:
            break
    index = index[np.argmin(X)]
    
#     sortmax = np.argsort(-cur_cpu)
#     index = sortmax.shape[0] - 1
#     for l in range(sortmax.shape[0]):
#         if cur_cpu[sortmax[l]] + ins_cpu[i]/100 <= 80:
#             index = l
#             break
#     LIST = []
#     if sortmax.shape[0] - index < 10:
#         LIST = sortmax[sortmax.shape[0]-10:sortmax.shape[0]]
#     else:
#         LIST = sortmax[index:index+10]
#     X = []
#     for key in LIST:
#         BED[cur_machine[key]][task_type[i]] += 1
#         a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
#         X.append(a)
#         BED[cur_machine[key]][task_type[i]] -= 1
#         if a == 1:
#             break
#     index = LIST[np.argmin(X)]

#     min_score = 11
# #     np.random.shuffle(random_list)
#     random_list = np.argsort(cur_score)
#     index = 0
#     for key in random_list:
# #         if cur_score[key] > cur_score[random_list[0]] and min_score != 11:
# #             break
#         if cur_cpu[key] + ins_cpu[i]/100 > 105:
#             continue
#         BED[cur_machine[key]][task_type[i]] += 1
#         a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
#         BED[cur_machine[key]][task_type[i]] -= 1
#         if a < min_score:
#             min_score = a
#             index = key
#             if a == 1:
#                 break
#     index = np.argmin(cur_score)


    cur_cpu[index] += ins_cpu[i]/100
    target = cur_machine[index]
    BED[target][task_type[i]] += 1
#     cur_score[index] = MODEL[target].predict(np.asarray(BED[target]).reshape(1, -1))[0]
#     print(cur_score[index], end=',')

    
    if end_time[i] not in cpu_delta:
        cpu_delta[end_time[i]] = np.zeros(cur_machine.shape[0])

    cpu_delta[end_time[i]][index] += ins_cpu[i]/100
       
    if end_time[i] not in end_list:
        end_list[end_time[i]] = {}
    if target not in end_list[end_time[i]]:
        end_list[end_time[i]][target] = []
    end_list[end_time[i]][target].append(task_type[i])
print(time.time()-s)    

In [ ]:
temp = []
for key in e_score:
    temp.append(e_score[key])
np.mean(temp)

In [ ]:
temp = []
for key in e_macpu:
    temp.append(e_macpu[key])
np.mean(temp)

In [ ]:
np.savetxt("/data/zkp/figure/simulation/spread_cpu.txt", np.asarray(temp))
# np.savetxt("/data/zkp/figure/simulation/spread_score.txt", np.asarray(temp), fmt="%d")

In [ ]:
temp = []
for key in e_BED:
    temp.append(e_BED[key])
print(np.asarray(temp).shape)
np.save("/data/zkp/figure/simulation/if2_BED", np.asarray(temp))

In [ ]:
index = 1
for key in e_score:
    print(e_score[key][index], e_macpu[key][index])

In [ ]:
temp = np.asarray(temp)

In [ ]:
np.mean(temp[:,0])

In [ ]:
for i in range(59):
    print(np.mean(temp[:, i]), np.mean(base[:, i]))

In [ ]:
new_P = [[], [], [], []]
new_T = [[], [], [], []]
for index in range(4):
    for i in range(len(P[index])):
        
        new_P[index].append(P[index][i][:348].tolist())
        new_T[index].append(T[index][i][:348].tolist())

In [ ]:
new_P = np.asarray(new_P)
new_T = np.asarray(new_T)

In [ ]:
predict = new_P
y = new_T
fontsize=20
plt.figure(figsize=(8, 7), dpi=120)
for i, n, ls in zip(range(y.shape[0]), ["w=1", "w=CPU quota", "w=Corr", "w=COV"], ["-", "--", "-.", ":"]):
    f1 = []
    for j in range(y.shape[1]):
        cnt = [[0, 0], [0, 0]]
        for k in range(predict.shape[2]):
            cnt[int(predict[i, j, k]<3)][int(y[i, j, k]>8)] += 1
        f1.append(cnt[1][1] / predict.shape[2])
    plt.plot(sorted(f1), np.arange(len(f1))*100 / (len(f1) - 1), label=n, lw=4, ls=ls)
plt.xlim(0, 0.1)
plt.xlabel("Proportion of severe prediction errors", fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.ylabel("CDF (%)", fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.ylim(0, 100)
plt.legend(fontsize=fontsize, loc="lower right")
plt.grid(ls=":")
# plt.savefig("./figure/ml-risk.png", bbox_inches="tight")
plt.show()

In [ ]:
temp = [2.4869791666666665,
2.4713541666666665,
2.4713541666666665,
2.4765625,
2.4322916666666665,
2.4270833333333335,
2.4479166666666665,
2.4140625,
2.4635416666666665,
2.4348958333333335,
2.3984375,
2.375,
2.375,
2.3541666666666665,
2.375,
2.3828125,
2.3932291666666665,
2.4140625,
2.4348958333333335,
2.4479166666666665,
2.3958333333333335,
2.4036458333333335,
2.4296875,
2.4036458333333335,
2.4375,
2.4348958333333335,
2.3958333333333335,
2.3854166666666665,
2.3619791666666665,
2.3645833333333335,
2.3984375,
2.3723958333333335,
2.3776041666666665,
2.4166666666666665,
2.4322916666666665,
2.4244791666666665,
2.4166666666666665,
2.4296875,
2.421875,
2.4375,
2.4166666666666665,
2.375,
2.3671875,
2.3645833333333335,
2.34375,
2.3515625,
2.34375,
2.3359375,
2.3463541666666665,
2.3359375,
2.3567708333333335,
2.3645833333333335,
2.375,
2.375,
2.3541666666666665,
2.3619791666666665,
2.4296875,
2.4401041666666665,
2.4296875,
2.4036458333333335]
np.mean(temp)

In [ ]:
df = data[(data.sample_time==1644501780)]
X = df.X.values
nc_sn = df.nc_sn.values
temp = []
for i in range(nc_sn.shape[0]):
    temp.append(MODEL[nc_sn[i]].predict(X[i].reshape(1,-1)))

In [ ]:
np.mean(df.machine_cpu), np.std(df.machine_cpu)